In [1]:
#%pip install statsmodels
import numpy as np
from statsmodels.stats.weightstats import ztest, ttest_ind
import statsmodels.api as sm
import statsmodels.stats.api as sms
import pandas as pd

import math
from scipy.stats import norm, chisquare, chi2

# Important statictical concepts for A/B testing
In this Notebook I have implemented and simulated on some important statistical concepts for experimentation. Hopefully this can help someone build intuition around those concepts.

# Risks in experiments
When we run experiments, one important goal is to bound the risks of making the incorrect decision. For any statistical test we have the intended risks, i.e., the values at wich we are trying to bound our risks, and the actual risks that we take in practice. If we fulfill all assumptions of a test (often large enough sample) and we have designed our experiment properly, we can choose the intended risks before the experiment, and then obtain these risk levels in practice. However, if we do not meet the assumptions of the test or somehow missuse the test, we may get much larger risks than intended. 

## False positive rate 
The intended false positive rate, often denoted as alpha is the rate at which we will detect an effect when in fact there is no effect. If we meet the assumptions of the test we are using, we obtain the intended false positve rate (or lower) over repeated experiments.

Below I run a simulation with large normal samples. Both the t-test and the z-test are valid and should return the intended false positive rates.

In [2]:
pvaluesZ = []
pvaluesT = []

reps = 1000
mean = 0
std_dev = 1
n = 1000
alpha = 0.05

for i in range(reps):
    y0 = np.random.normal(mean, std_dev, n)
    y1 = np.random.normal(mean, std_dev, n)
    
    stat, pvalue = ztest(x1=y1, x2=y0, alternative='two-sided')
    pvaluesZ.append(pvalue<=alpha)
    
    res = ttest_ind(x1=y1, x2=y0, alternative='two-sided')
    pvaluesT.append(res[1]<=alpha)
    
print(np.mean(pvaluesZ))
print(np.mean(pvaluesT))

0.041
0.041


For this sample size the false postive rate is the same for t and z, both close to alpha, as expected.

### Multiple testing correction
If we have more than 1 metric (or several treatment groups) we have more than 1 chance to get a false positive result. A multiple testing correction counters this false positive rate inflation. The most common method is Bonferroni where we simply divide the intended false postive rate alpha by the number of metrics (or more generally the number of tests).

In [3]:
sig_no_bon = []
sig_bon = []

reps = 1000
n_samples = 1000
alpha = 0.05

for i in range(reps):
    df_experiment = pd.DataFrame({
    'Metric1': np.random.normal(1, 2, n_samples*2),
    'Metric2': np.random.normal(2, 2, n_samples*2),
    'Metric3': np.random.normal(10, 2, n_samples*2),
    'group': np.random.binomial(1,0.5,n_samples*2)
    })

    stat_1, pvalue_1 = ztest(x1=df_experiment[df_experiment['group']==0]["Metric1"], x2=df_experiment[df_experiment['group']==1]["Metric1"], alternative='two-sided')
    stat_2, pvalue_2 = ztest(x1=df_experiment[df_experiment['group']==0]["Metric2"], x2=df_experiment[df_experiment['group']==1]["Metric2"], alternative='two-sided')
    stat_3, pvalue_3 = ztest(x1=df_experiment[df_experiment['group']==0]["Metric3"], x2=df_experiment[df_experiment['group']==1]["Metric3"], alternative='two-sided')

    alpha_bon_adjust = alpha/3
    sig_no_bon.append(any([pvalue_1<=alpha, pvalue_2<=alpha, pvalue_3<=alpha]))
    sig_bon.append(any([pvalue_1<=alpha_bon_adjust, pvalue_2<=alpha_bon_adjust, pvalue_3<=alpha_bon_adjust]))
        
print(f'False positive rate without mutliple testing correction: {np.mean(sig_no_bon)}')
print(f'False positive rate with Bonferroni correction: {np.mean(sig_bon)}')


False positive rate without mutliple testing correction: 0.164
False positive rate with Bonferroni correction: 0.056


As we can see, with 3 independent metrics the false positive rate is inflated a lot for finding at least one significant metric. The Bonferroni correction bounds the false positive rate below alpha as intended. 

### Repeated testing (peeking)
If we are looking at the statistical results during the data collection (for example to abort early if harm is detected) we have repeated chances to find a false positive result. We can correct for the false positive rate inflation by using so called sequential test. In principle, sequential tests are similar to multiple testing correction. The difference is that for repeated testing we know that the tests are correlated (as we are measuring repeatedly on the same increasing sample) and can therefore do more efficient corrections, which is what sequential tests are doing.


Below is a simulation with repeated testing and no correction, Bonferroni, and always valid sequantial tests.

In [4]:
def ci(alpha, ro, n):
    # This is an always valid CI from https://arxiv.org/pdf/2302.10108.pdf?fbclid=IwAR11HKKqFiMUinXj2NrvpOA0KQPAJPpbRTS0SPMQj5W82gsqctnwESw0hHY
    return math.sqrt(2 * (n * ro**2 + 1)/(n**2 * ro**2) * math.log(math.sqrt(n * ro**2 + 1) / alpha))

In [5]:
# Check false positive rate over repeated testing with different methods
sig_no_corr = []
sig_bonferroni = []
sig_corr = []
n_samples = 1000
alpha = 0.05
T = 10 # Number of repeated tests per sample
ro = 1
p = 0.5
for j in range(500):
    result_df = pd.DataFrame({'metric': np.random.normal(10, 2.5, n_samples*2), 'group': np.random.binomial(1, 0.5, n_samples*2)})
    X = sm.add_constant(result_df[['group']])
    y = result_df['metric']
    Z = ((result_df['group']/p) - ((1 - result_df['group'])/(1 - p))) * y
    sig_temp_no_corr = []
    sig_temp_bonferroni = []
    sig_temp_corr = []
    for i in range(T):
        curr_n = int(n_samples*2*((i+1)/T))

        # with standard inference
        model = sm.OLS(y.head(curr_n), X.head(curr_n)).fit()
        sig_temp_no_corr.append(model.summary2().tables[1]['P>|t|'].iloc[1]<=alpha)

        # with standard inference and Bonferroni
        sig_temp_bonferroni.append(model.summary2().tables[1]['P>|t|'].iloc[1]<=alpha/T)
        
        # with always valid inference
        sig_temp_corr.append(Z.head(curr_n).mean() - math.sqrt(np.var(Z.head(curr_n))) * ci(alpha, ro, curr_n) >= 0 or Z.head(curr_n).mean() + math.sqrt(np.var(Z.head(curr_n))) * ci(alpha, ro, curr_n) <= 0)  

    sig_no_corr.append(any(sig_temp_no_corr))
    sig_bonferroni.append(any(sig_temp_bonferroni))
    sig_corr.append(any(sig_temp_corr))
    
print(f'False positive rate under repeated testing with standard inference: {np.mean(sig_no_corr)}')
print(f'False positive rate under repeated testing with Bonferroni correction: {np.mean(sig_bonferroni)}')
print(f'False positive rate under repeated testing with always valid inference: {np.mean(sig_corr)}')


False positive rate under repeated testing with standard inference: 0.198
False positive rate under repeated testing with Bonferroni correction: 0.032
False positive rate under repeated testing with always valid inference: 0.002


We can see that with standard inference the false positive rate is very inflated compared to the intented alpha of 0.05. Bonferroni is bounding the false positive rate as expected, it is conservative because it does not take the correlation into account. With always valid inference the false positive rate is far below alpha. This is expected since the always valid test corrects for looking after each user coming in and we only look 10 times for 2000 users.

# False negative rate and sample size calculation
The intended false negative rate, often denoted beta (which is 1 minus the intended power), is the rate at which we don't detect a certain treatment effect when it in fact exists. If we design our experiment such that we meet the required sample size to achieve the power, and we fulfill all other assumptions of the test, we obtain the intended false negative rate (or lower) over repeated experiments.

In [6]:
# Using a package
alpha = 0.05
beta = 0.2
ratio = 1
effect_size = 0.1
N1 = sms.NormalIndPower().solve_power(effect_size=effect_size, nobs1=None, alpha=alpha, power=1-beta, ratio=ratio, alternative='two-sided')
N2 = N1*ratio
print(round(N1+N2))

3140


In [7]:
# Manual implementation
def required_sample_size_z_test(
    alpha=0.05, beta=0.2, variance=1, abs_MDE=0.1, treat_p=0.5
):
    kappa = treat_p / (1 - treat_p)
    nB = (
        (norm.ppf(1 - alpha) + norm.ppf(1 - beta)) ** 2
        / abs_MDE**2
        * variance
        * (1 + 1 / kappa)
    )
    N = nB * (1 + kappa)
    return int(np.ceil(N))

In [8]:
manual_ss = required_sample_size_z_test(alpha=alpha/2, beta=beta, variance=1, abs_MDE=0.1, treat_p=0.5)
print(manual_ss)

3140


In [9]:
# Check package and I agree
manual_ss == round(N1+N2)

True

Below is a simulation were we have powered the experiment to 80% for the true treatment effect.

In [10]:
sigZ = []

reps = 1000
effect = 0.1
mean = 0
std_dev = 1
n = manual_ss//2
alpha = 0.05

for i in range(reps):
    y0 = np.random.normal(mean, std_dev, n)
    y1 = np.random.normal(mean, std_dev, n) + effect
    
    stat, pvalue = ztest(x1=y1, x2=y0, alternative='two-sided')
    sigZ.append(pvalue<=alpha)
    
print(f"The power is {round(np.mean(sigZ)*100,2)}%.")


The power is 81.2%.


If indeed we use the required sample size returned by the sample size caluclation, and we generate data with the hypothesised effect, we obtain the intented power over repeated experiments.

# Experiment efficiency and Experimental design
There are essentially two steps to any experimental design. The sampling, i.e., which subset of the population that is in our sample, and, the treatment assignment, i.e., which users in our sample that is in treatment and control, respectively (sticking to two groups here for simplicity).

## Sampling design
Sampling design is about making the sample as representative of the population as possible. There are two main reasons for not just drawing a random sample from the population:
1. There is a risk that by chance certain types of users are not represented in the sample (inclusion).
2. We can reduce the variance in our estimators over repeated sampling by ensuring that our sample is a good representation of the population. 

Sampling design can be done before or after the experiment. The most common pre-experiment sampling design is stratified sampling, where the population is split into important substrata and we are taking random samples from each strata (proportional to their relative size) to ensure all strata are represented in the sample. It is also possible to do sampling design after the experiment. The most common method is called post stratification. In post-stratification we are using our knowledge about the true proportion of the strata in the population to re-weight the treatment effect estimator. For example, is there is 50/50 young/old in the population but we have 70/30 in our sample we would down-weight the young users and up-weight the old users in our estimator to obtain an unbiased population average treatment effect estimator, even though our sampling is biased. 

### Practical concerns
In online experimentation, we can typically not control sampling. That is, users come in whenever they are active and we are usually not willing to wait for certain users for too long. Moreover, stratified sampling is somewhat complicated (at least costly) to achieve technically in a feature-flagging system. For this reason post-stratification is the most plausible alternative. Post stratification can of course not solve the issue of some users not being represented at all in the sample, however, in large samples it is very unlikley that groups that are not too small in the population are not at all represented in the sample. 

## Treatment assignment design
Treatment assignment design is all about restricting the random treatment assignment to not allow treatment assignments that we know are not giving similar groups in terms of the outcome. Of course we do not know the outcome at the design stage, but if we have covariates that are correlated with the outcome, we know that imbalance in the covariates is likley to imply imbalance in the outcome.  

Similar to sampling design, there are two main reasons for restricting the randomization with a design: 
 1. There is a risk that by chance certain types of users are not represented in all treatment groups (inclusion).
 2. We can reduce the variance in our estimators over repeated experimenting by ensuring that our treatment groups are as similar as possible in terms of covariates that are correlated with the outcome.



 Treatment assignment design can be done before and/or after the experiment. The most common pre-experiment treatment assignment designs are stratified randomization and rerandomization. The most common post-experiment design is regression adjustment (also called cuped). 



### Rerandomization
Stratified randomization is a special case of rerandomization, so I stick to rerandomization here to exemplify what I know about design. 
Rerandomization is simply re-randomizing the treatment assignment until the groups are similar enough on some similarity measure. Since this decreases the variance of the estimator, the statistical inference must be adjusted accordingly. 

In [11]:

reps = 1000
mean = 0
std_dev = 1
n = 2000
alpha = 0.05
rerand_n = 20

std_err_ratio = []
sigZ = []
sigOLS = []
sigRR_Z = []
sigRR_OLS = []
estimates_no_RR = []
estimates_no_RR_OLS = [] 
estimates_RR = []

for i in range(reps):
    # generating data
    x = np.random.normal(mean, std_dev, n*2)
    y = x + np.random.normal(mean, std_dev, n*2)
    treat = np.random.binomial(1,0.5,n*2) 

    # Without rerandomization
    ## standard z-test
    stat, pvalue = ztest(x1=y[treat==1], x2=y[treat==0], alternative='two-sided')
    sigZ.append(pvalue<=alpha)
    estimates_no_RR.append(np.mean(y[treat==1]) - np.mean(y[treat==0]))

    # using OLS (standard variance reduction)
    X = np.column_stack((x,treat))
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    sigOLS.append(model.pvalues[2]<=alpha)
    estimates_no_RR_OLS.append(model.params[2])

    # rerandomize
    diff = float('inf')
    for r in range(rerand_n):
        treat_temp = np.random.binomial(1,0.5,n*2) 
        diff_temp = (np.mean(x[treat_temp==1])-np.mean(x[treat_temp==0]))**2
        if diff_temp < diff:
            treat = treat_temp
            diff = diff_temp
        
    # inference after rerandomization

    ## Not taking rerandomization into account
    stat, pvalue = ztest(x1=y[treat==1], x2=y[treat==0], alternative='two-sided')
    sigRR_Z.append(pvalue<=alpha)
    
    ## Taking rerandomization into account
    X = np.column_stack((x,treat))
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    sigRR_OLS.append(model.pvalues[2]<=alpha)
    estimates_RR.append(model.params[2])

In [12]:
print(f"The false positive rate for z test without rerandomization:{np.mean(sigZ)}")
print(f"The false positive rate for z test with rerandomization {np.mean(sigRR_Z)}")
print(f"The false positive rate for OLS-z-test without rerandomization {np.mean(sigOLS)}")
print(f"The false positive rate for OLS-z-test with rerandomization {np.mean(sigRR_OLS)}")

The false positive rate for z test without rerandomization:0.055
The false positive rate for z test with rerandomization 0.003
The false positive rate for OLS-z-test without rerandomization 0.048
The false positive rate for OLS-z-test with rerandomization 0.045


As we have seen before, using a z test without rerandomization works as intended. Using rerandomization in combinaition with a standard z test (that doesn't take the rerandomization into account) gives a very conservative test, as the rerandomization removes some of the variation. One valid inference approach after rerandomization is to use OLS with the same covariate as in the rerandomization. As we can see rerandomization combined with OLS give the right false positive rate.

Of course, an alternative used by many online experimenters is variance reduction, that is only OLS with the covariate after the experiment. As expected this too gives the right false positive rate.

In [13]:
print(f"The variance of the estimator with rerandomization is {np.round((np.var(estimates_no_RR) - np.var(estimates_RR))/ np.var(estimates_no_RR) * 100,2 ) } % smaller than without" )
print(f"The variance of the estimator with only post-variance reduction is {np.round((np.var(estimates_no_RR) - np.var(estimates_no_RR_OLS))/ np.var(estimates_no_RR) * 100,2 ) } % smaller than without" )


The variance of the estimator with rerandomization is 52.54 % smaller than without
The variance of the estimator with only post-variance reduction is 51.92 % smaller than without


In terms of variance reduction, reandomization in combination with OLS on one hand, and only using OLS on the other, are comparable. In large sample, the reduction will be the same. This indicated that for large sample sizes the only reason for doing it at the design stage is the inclusion argument discussed above.

Source: https://academic.oup.com/jrsssb/article/82/1/241/7056044?fbclid=IwAR1br3pbcE0XrJOmQF8T-PJAPQmnSUvCPazw_Nlxed6RMODbyN-dLquYrLM&login=false

# Other tests
In online A/B testing almost all tests are mean based which means that we can use z and t test in large samples. There are some exceptions. 

## Chi-square
One test that almost all experientation tools provide is the chi square test, which in this context is used for sample ratio mismatch, which is an experiment validity test.

Below is simulation testing two proportions over repeated randomization.

In [14]:
sigChiSquare = []
sigManualChiSquare = []

reps = 1000
mean = 0
std_dev = 1
n = 1000
alpha = 0.05

for i in range(reps):
    treat = np.random.binomial(1, 0.5, n*2)
    
    # Using a package
    res = chisquare([np.sum(treat==1),np.sum(treat==0)], f_exp=[n,n])
    sigChiSquare.append(res.pvalue<=alpha)

    # Manually
    chisqr = (np.sum(treat==1)-n)**2/n + (np.sum(treat==0)-n)**2/n
    pvalue = 1 - chi2.cdf(chisqr, 1)
    sigManualChiSquare.append(pvalue<=alpha)
    
print(np.mean(sigChiSquare))
print(np.mean(sigManualChiSquare))
sigChiSquare == sigManualChiSquare

0.046
0.046


True

As we can see the test has the intended false positive rate.
## Bootstrap
An inference method that is useful for estimators where the sampling distribution is hard to obtain is bootstrap. Bootstrap is a resampling method for appoximating the sampling distributuon of any estimator. Below I have implemented it for the difference-in-medians estimator. Bootstrap is computationaly complex but there are methods to make it more efficient for example poisson bootstrap. 

In [15]:
def bootstrap_median_diff_ci(y, treat,alpha, b):
    y1 = y[treat==1]
    n1 = np.sum(treat==1) 
    y0 = y[treat==0]
    n0 = np.sum(treat==0)
    diff = []
    for i in range(b):
        diff.append(np.median(np.random.choice(y1, n1, replace=True))-np.median(np.random.choice(y0, n0, replace=True)))

    return np.quantile(diff, [alpha/2, 1-alpha/2])


alpha = 0.1
reps = 1000
n_samples = 100
sig = []
for i in range(reps):
    temp_ci = bootstrap_median_diff_ci(np.random.normal(1,1,n_samples), np.random.binomial(1,0.5, n_samples),alpha = alpha, b=100)
    sig.append(temp_ci[0] > 0 or temp_ci[1]<0)
    
print(f"The coverage of the bootstrap CI for the diffeence-in-medians is {(1-np.mean(sig)) * 100}%")

The coverage of the bootstrap CI for the diffeence-in-medians is 89.1%


As we can see the coverage is close to the intended 90% already with 100 bootstrap samples.